In [ ]:
import threading
import xmlrpc
from xmlrpc.server import SimpleXMLRPCServer, SimpleXMLRPCRequestHandler
from xmlrpc.client import ServerProxy, Fault
import datetime
import time
from collections import deque


class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)

proxy_server = SimpleXMLRPCServer(("127.0.0.1", 8028), requestHandler=RequestHandler, allow_none=True)

# main_server = xmlrpc.client.ServerProxy("http://127.0.0.1:8008", allow_none=True)

workers = {}
worker_list = deque()
lock = threading.Lock()


stats_server = xmlrpc.client.ServerProxy("http://127.0.0.1:8018", allow_none=True)

# Rate limiting: N запросов в T секунд
RATE_LIMIT_N = 5
RATE_LIMIT_T = 5
# Храним timestamps последних запросов
request_times = deque()


def set_rate_limit(n, t):
    global RATE_LIMIT_N, RATE_LIMIT_T
    RATE_LIMIT_N = n
    RATE_LIMIT_T = t
    return True


proxy_server.register_function(set_rate_limit, 'set_rate_limit')

# Логгируем
def log_event(event_type, server_addr=None, duration=None):
    try:
        timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        stats_server.add_log(event_type, timestamp, duration, server_addr or "proxy")
    except Exception as e:
        print("Не удалось записать лог:", e)

# + воркер
def register_server(ip, port, heartbeat_interval=15):
    start_time = time.time()
    addr = f"{ip}:{port}"
    print("hi ", addr)
    with lock:
        if addr not in workers:
            print(f"[REGISTRY] Новый сервер: {addr}")
            try:
                worker_proxy = ServerProxy(f"http://{addr}", allow_none=True)
                workers[addr] = {
                    "proxy": worker_proxy,
                    "last_seen": time.time(),
                    "missed": 0,
                    "heartbeat_interval": heartbeat_interval
                }
                worker_list.append(addr)
                log_event("server_registered", addr)
            except:
                return False
        else:
            # Обновляем время жизни
            workers[addr]["last_seen"] = time.time()
            workers[addr]["missed"] = 0
        return True
proxy_server.register_function(register_server, "register_server")

def heartbeat(ip, port):
    addr = f"{ip}:{port}"
    with lock:
        if addr in workers:
            workers[addr]["last_seen"] = time.time()
            workers[addr]["missed"] = 0
            return True
    return False
proxy_server.register_function(heartbeat, "heartbeat")

#
def cleanup_dead_workers():
    while True:
        time.sleep(10)
        now = time.time()
        with lock:
            dead = []
            for addr, info in workers.items():
                expected = info["last_seen"] + info["heartbeat_interval"] * 3
                if now > expected:
                    info["missed"] += 1
                    if info["missed"] >= 3:
                        dead.append(addr)
            for addr in dead:
                print(f"[REGISTRY] Сервер умер: {addr}")
                workers.pop(addr, None)
                worker_list.remove(addr)
                log_event("server_died", addr)

threading.Thread(target=cleanup_dead_workers, daemon=True).start()

# Round Robin
def get_next_worker():
    with lock:
        if not worker_list:
            raise Exception("Нет доступных воркеров")
        addr = worker_list[0]
        worker_list.rotate(-1)
        return workers[addr]["proxy"], addr

# Прокся функция с логгированием и РЛ
def proxy_function(*args, **kwargs):
    method_name = proxy_function.method_name

    # === Rate Limiting ===
    current_time = time.time()
    while request_times and request_times[0] < current_time - RATE_LIMIT_T:
        request_times.popleft()

    if len(request_times) >= RATE_LIMIT_N:
        # === ЛИМИТ СРАБОТАЛ ===
        wait_start = time.time()
        wait_until = request_times[0] + RATE_LIMIT_T
        wait_time = wait_until - current_time

        # Ждём
        time.sleep(wait_time)

        # === ЛОГИРУЕМ ОЖИДАНИЕ ===
        event_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        try:
            stats_server.add_log('too_many_requests', event_time, round(wait_time, 3))
        except (Fault, ConnectionRefusedError, Exception):
            pass  # Игнорируем, если stats недоступен

        # Обновляем очередь после ожидания
        current_time = time.time()
        while request_times and request_times[0] < current_time - RATE_LIMIT_T:
            request_times.popleft()

    # Добавляем текущий запрос
    request_times.append(time.time())

    # === Выполнение метода ===
    start_time = time.time()
    event_time_str = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    try:
        worker_proxy, addr = get_next_worker()
        method = getattr(worker_proxy, method_name)
        result = method(*args, **kwargs)

        end_time = time.time()
        duration = end_time - start_time

        # Лог успешного вызова
        try:
            log_event(method_name, addr, round(duration, 3))
        except (Fault, ConnectionRefusedError, Exception):
            pass

        return result
    except Fault as e:
        print(f"Ошибка XML-RPC при вызове метода {method_name}: {e}")
        raise
    except Exception as e:
        print(f"Неизвестная ошибка при вызове метода {method_name}: {e}")
        raise
    except:
        duration = time.time() - start_time
        log_event(f"{method_name}_failed", "no_worker", round(duration, 4))
        raise Fault(1, "Нет доступных серверов")


# Регистрируем все методы
methods = ['ping', 'now', 'type', 'sum', 'pow', 'black_list_check', 'black_list_check_full',
           'send_back_binary', 'color_inversion', 'send_back_binarization',
           'send_back_binarization_with_percent', 'send_back_flip_vertical']

for method in methods:
    def create_proxy_method(method_name):
        def specific_proxy(*args, **kwargs):
            proxy_function.method_name = method_name
            return proxy_function(*args, **kwargs)

        return specific_proxy


    proxy_server.register_function(create_proxy_method(method), method)

proxy_server.register_introspection_functions()

def ping_proxy():
    return True
proxy_server.register_function(ping_proxy, "ping_proxy")

print("Listening PROXY on port 8028...")
proxy_server.serve_forever()

Listening PROXY on port 8028...
hi  127.0.0.1:8008
[REGISTRY] Новый сервер: 127.0.0.1:8008
hi  127.0.0.1:8007
[REGISTRY] Новый сервер: 127.0.0.1:8007
hi  127.0.0.1:8006
[REGISTRY] Новый сервер: 127.0.0.1:8006


127.0.0.1 - - [29/Nov/2025 23:25:45] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2025 23:25:45] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2025 23:25:45] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2025 23:25:45] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2025 23:25:45] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2025 23:25:45] "POST /RPC2 HTTP/1.1" 200 -


hi  127.0.0.1:8008
hi  127.0.0.1:8006
hi  127.0.0.1:8007


127.0.0.1 - - [29/Nov/2025 23:25:55] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2025 23:25:55] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2025 23:25:55] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2025 23:25:55] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2025 23:25:55] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2025 23:25:55] "POST /RPC2 HTTP/1.1" 200 -


[REGISTRY] Сервер умер: 127.0.0.1:8008
[REGISTRY] Сервер умер: 127.0.0.1:8007
[REGISTRY] Сервер умер: 127.0.0.1:8006
hi  127.0.0.1:8008
[REGISTRY] Новый сервер: 127.0.0.1:8008


...

127.0.0.1 - - [29/Nov/2025 23:40:28] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2025 23:40:28] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2025 23:40:29] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2025 23:40:29] "POST /RPC2 HTTP/1.1" 200 -


hi  127.0.0.1:8006
hi  127.0.0.1:8007


127.0.0.1 - - [29/Nov/2025 23:40:39] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2025 23:40:39] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2025 23:40:39] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2025 23:40:39] "POST /RPC2 HTTP/1.1" 200 -


hi  127.0.0.1:8007
hi  127.0.0.1:8006


127.0.0.1 - - [29/Nov/2025 23:40:49] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2025 23:40:49] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2025 23:40:49] "POST /RPC2 HTTP/1.1" 200 -
127.0.0.1 - - [29/Nov/2025 23:40:49] "POST /RPC2 HTTP/1.1" 200 -
